In [168]:
import pandas as pd 
import numpy as np 

In [169]:
data = pd.read_csv('training_data.csv')
data.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,14/3/11,Roman,1390,Roman,34.938093,-9.856322,none,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,6/3/13,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,25/2/13,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,28/1/13,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,13/7/11,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [170]:
data['gov_funded'] = 0
data['gov_funded'][data['funder'] == 'Government Of Tanzania'] = 1

pd.options.mode.chained_assignment = None  # default='warn'


In [171]:
print data['gov_funded'].value_counts()

0    50316
1     9084
Name: gov_funded, dtype: int64


In [172]:
#re-cat waterpoint 
data_waterpoint = pd.get_dummies(data['waterpoint_type_group'])
data_waterpoint.head()

#re-cat quality group 
data_quality = pd.get_dummies(data['quality_group'])
data_quality = data_quality.drop('fluoride', 1)
data_quality = data_quality.drop('colored', 1)
data_quality = data_quality.drop('milky', 1)
data_quality = data_quality.drop('salty', 1)
data_quality = data_quality.drop('unknown', 1)


data_quality=data_quality.rename(columns = {'good':'good_quality'})
data_quality.head()

print data_quality['good_quality'].value_counts()



1    50818
0     8582
Name: good_quality, dtype: int64


In [173]:
#recat status_group 

vals_to_replace = {'functional': 0, 'functional needs repair': 1, 'non functional': 1}
data['status_group'] = data['status_group'].map(vals_to_replace)


In [174]:
#drop unused columns
cols_to_drop = [['date_recorded'], ['gps_height']]
data = data.drop(['date_recorded'], axis=1)
data = data.drop(['num_private'], axis=1)

data['construction_year']

0        1999
1        2010
2        2009
3        1986
4           0
5        2009
6           0
7           0
8           0
9           0
10       2011
11       1987
12          0
13       2009
14          0
15       1991
16       1978
17       1978
18       1999
19       1992
20       2008
21          0
22       1978
23       2011
24       2009
25       1974
26       2011
27          0
28       2000
29       2002
         ... 
59370       0
59371       0
59372       0
59373    1988
59374       0
59375    2005
59376    1990
59377       0
59378    2008
59379       0
59380    1999
59381    1992
59382    2008
59383    2008
59384       0
59385    2011
59386    2009
59387    2000
59388    1995
59389    2009
59390    1991
59391    1967
59392       0
59393       0
59394    2007
59395    1999
59396    1996
59397       0
59398       0
59399    2002
Name: construction_year, dtype: int64

In [175]:

#concat the new datasets 
df_new = pd.concat([data, data_waterpoint, data_quality], axis=1)


df_new['population']

0         109
1         280
2         250
3          58
4           0
5           1
6           0
7           0
8           0
9           0
10        345
11        250
12          0
13          1
14          0
15        200
16         35
17         50
18       1000
19          1
20          4
21          0
22        350
23        210
24        156
25        140
26        260
27          0
28          1
29          1
         ... 
59370       1
59371       0
59372       0
59373      96
59374       0
59375     609
59376       1
59377       0
59378      36
59379       0
59380      50
59381     360
59382       1
59383     800
59384       0
59385     200
59386    1000
59387     100
59388     500
59389    1500
59390     150
59391     210
59392       0
59393       0
59394      89
59395     125
59396      56
59397       0
59398       0
59399     150
Name: population, dtype: int64

In [176]:
y = data['status_group']
cols_to_use = ['gov_funded', 'population', 'amount_tsh', 'good_quality', 'cattle trough', 'communal standpipe', 'dam', 'hand pump', 'improved spring', 'construction_year', 'population', 'longitude', 'latitude', 'amount_tsh']
X = df_new[cols_to_use]


In [177]:
# Splitting the dataset into the Training set and Test set
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [178]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense

In [179]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu', input_dim = 14))

# Adding the second hidden layer
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, nb_epoch = 100)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(kernel_initializer="uniform", activation="relu", input_dim=14, units=6)`
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=6, activation="relu", kernel_initializer="uniform")`
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1, activation="sigmoid", kernel_initializer="uniform")`


Epoch 1/100
47520/47520 [==============================] - 19s - loss: 0.6301 - acc: 0.6377    
Epoch 2/100
47520/47520 [==============================] - 19s - loss: 0.6183 - acc: 0.6442    
Epoch 3/100
47520/47520 [==============================] - 17s - loss: 0.6149 - acc: 0.6461    
Epoch 4/100
47520/47520 [==============================] - 19s - loss: 0.6120 - acc: 0.6519    
Epoch 5/100
47520/47520 [==============================] - 19s - loss: 0.6098 - acc: 0.6556    
Epoch 6/100
47520/47520 [==============================] - 19s - loss: 0.6079 - acc: 0.6594    
Epoch 7/100
47520/47520 [==============================] - 19s - loss: 0.6064 - acc: 0.6613    
Epoch 8/100
47520/47520 [==============================] - 19s - loss: 0.6057 - acc: 0.6636    
Epoch 9/100
47520/47520 [==============================] - 19s - loss: 0.6048 - acc: 0.6655    
Epoch 10/100
47520/47520 [==============================] - 19s - loss: 0.6043 - acc: 0.6661    
Epoch 11/100
47520/47520 [=============

In [219]:
from __future__ import division

# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print cm 

ANN_accuracy = (5059 + 2849) / (5059 + 1385 + 2587 + 2849)
print ANN_accuracy 

[[5059 1385]
 [2587 2849]]
0.665656565657


In [220]:
scores = classifier.evaluate(X_test, y_test)
print scores 

10976/11880 [==========================>...] - ETA: 0s[0.59764978821430137, 0.66565656565656561]


In [221]:
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X_train, y_train)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [222]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

# Applying k-Fold Cross Validation
from sklearn.cross_validation import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, scoring='accuracy')
print accuracies.mean()
print accuracies.std()


0.703282974566
0.00463340860988


In [223]:
print "XGBoost accuracy: %.1f" % (accuracies.mean()*100) + "%"
print "ANN accuracy: %.1f" % (ANN_accuracy*100) + "%"

XGBoost accuracy: 70.3%
ANN accuracy: 66.6%
